# Import comet_ml
- need to be on the top, otherwise error
- comet_ml is a library that helps us to track the experiement hyperparameters and metrics

In [22]:
# import comet_ml at the top of your file
from comet_ml import Experiment
experiment = Experiment(
            project_name="music-genre-multiclass-classification",
            workspace="wodenwang820118",
        )
import comet_ml
import logging

COMET INFO: Optimizer metrics is 'loss' but no logged values found. Experiment ignored in sweep.
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/709f64d91f2440f4a22190c79251ab90
COMET INFO:   Others:
COMET INFO:     optimizer_count        : 1
COMET INFO:     optimizer_id           : ad5cb844a2d4484c8868ba5548efddea
COMET INFO:     optimizer_metric       : loss
COMET INFO:     optimizer_metric_value : 1
COMET INFO:     optimizer_objective    : minimum
COMET INFO:     optimizer_parameters   : {"batch_size": 32, "epochs": 9, "first_layer_units": 1000, "second_layer_units": 100, "third_layer_units": 32}
COMET INFO:     optimizer_pid          : 4ee46acef5369a8b332a98c068614a7debfb4fc1
COMET INFO:     optimizer_process      : 10548
COMET INFO:    

In [23]:
logging.basicConfig(level=logging.INFO)
LOGGER = logging.getLogger("comet_ml")

# Import libraries

In [24]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow import keras

%matplotlib inline
sns.set_style('whitegrid')

In [25]:
# tensorflow 2.7 
import tensorflow as tf 
from tensorflow.keras.layers import Input,Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Model,Sequential

In [26]:
# scale the numeric data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Import data and pre-preocess data

In [27]:
df_train_features = pd.read_csv('train_features.csv')
df_train_labels = pd.read_csv('train_labels.csv')

df_valid_features = pd.read_csv('valid_features.csv')
df_valid_labels = pd.read_csv('valid_labels.csv')

df_test_features = pd.read_csv('test_features.csv')

In [28]:
num_train_data = df_train_features.iloc[:,9:]
num_valid_data = df_valid_features.iloc[:,9:]
num_test_data = df_test_features.iloc[:,9:]

In [29]:
num_train = scaler.fit_transform(num_train_data)
num_valid = scaler.fit_transform(num_valid_data)
num_test = scaler.fit_transform(num_test_data)

In [30]:
num_train_y = pd.get_dummies(df_train_labels['genre'])
num_valid_y = pd.get_dummies(df_valid_labels['genre'])

In [31]:
def str_remove_comma(string):
    str = string.split(", ")
    return str

def list_to_string(list):
    str = " "
    return str.join(list)

In [32]:
x_train = df_train_features['tags'].apply(lambda x : list_to_string(str_remove_comma(x)))
y_train = df_train_labels['genre'].to_numpy()

x_test = df_valid_features['tags'].apply(lambda x : list_to_string(str_remove_comma(x)))
y_test = df_valid_labels['genre'].to_numpy()

In [33]:
x_train = np.array(x_train)
x_test = np.array(x_test)

# Adjust weights and define early stop mechanism

In [34]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(df_train_labels['genre']),y=df_train_labels['genre'])
class_weight = {i :class_weight[i] for i in range(8)}

In [35]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',patience=5)

# Import ktrain for text classification model

In [36]:
import ktrain
from ktrain import text

# Text classification model

In [37]:
trn2, val2, preproc2 = text.texts_from_array(
    x_train=x_train, y_train=y_train,
    x_test=x_test, y_test=y_test,
    ngram_range=1, 
    maxlen=150,
    preprocess_mode='standard'
)

language: en
Word Counts: 4888
Nrows: 7678
7678 train sequences
train sequence lengths:
	mean : 80
	95percentile : 139
	99percentile : 193
x_train shape: (7678,150)
y_train shape: (7678, 8)
Is Multi-Label? False
450 test sequences
test sequence lengths:
	mean : 74
	95percentile : 142
	99percentile : 180
x_test shape: (450,150)
y_test shape: (450, 8)
task: text classification


In [38]:
model2 = text.text_classifier('fasttext', train_data=trn2, preproc=preproc2, metrics=['accuracy'])

Is Multi-Label? False
compiling word ID features...
maxlen is 150
done.


## The Dataset class for binding the audio data and the text data

In [39]:
import math
class MyCustomDataset(ktrain.SequenceDataset):
    def __init__(self, x, y, batch_size=32, shuffle=True):
        # error checks
        err = False
        if type(x) == np.ndarray and len(x.shape) != 2: err = True
        elif type(x) == list:
            for d in x:
                if type(d) != np.ndarray or len(d.shape) != 2:
                    err = True
                    break
        else: err = True
        if err:
            raise ValueError('x must be a 2d numpy array or a list of 2d numpy arrays')
        if type(y) != np.ndarray:
            raise ValueError('y must be a numpy array')
        if type(x) == np.ndarray:
            x = [x]
        # set variables

        super().__init__(batch_size=batch_size)
        self.x, self.y = x, y
        self.indices = np.arange(self.x[0].shape[0])
        self.n_inputs = len(x)
        self.shuffle = shuffle

    # required for instances of tf.keras.utils.Sequence
    def __len__(self):
        """Number of batches in the Sequence.
        """
        return math.ceil(self.x[0].shape[0] / self.batch_size)

    # required for instances of tf.keras.utils.Sequence
    def __getitem__(self, idx):
        """Gets batch at position `idx`.
        """
        # the starting and ending indices of the batch
        inds = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_x = []

        for i in range(self.n_inputs):
            batch_x.append(self.x[i][inds])
        batch_y = np.array(self.y[inds])
        return tuple(batch_x) , batch_y

    # required for instances of ktrain.Dataset
    def nsamples(self):
        return self.x[0].shape[0]

    #required for instances of ktrain.Dataset
    def get_y(self):
        return self.y

    def on_epoch_end(self):
        if self.shuffle:  np.random.shuffle(self.indices)

In [40]:
train_data = MyCustomDataset([num_train] + [trn2[0]], trn2[1])
valid_data = MyCustomDataset([num_valid] + [val2[0]], val2[1])

# Combined model
- the model takes the audio data and the combined train_data, valid_data.
- the purpose of the class is to grid search the best hyperparameters.

In [41]:
class AudioTextGenreClassifier:
    def __init__(self,num_train, valid_train, train_data,num_train_y,valid_data,num_valid_y,class_weight,early_stop,experiment,text_model):
        # num_train data for sequential model input shape
        self.num_train = num_train
        self.valid_train = valid_train
        
        # customed dataset
        self.train_data = train_data
        self.valid_data = valid_data
        self.num_train_y = num_train_y
        self.num_valid_y = num_valid_y
        # text model and data inputs
        self.text_model = text_model

        # class weights and early stopping
        self.class_weight = class_weight
        self.early_stop = early_stop

        # comet experiment
        self.experiment = experiment
    
    def build_model(self):
        # Build the model
        model = Sequential()
        model.add(
            Dense(
                self.experiment.get_parameter("first_layer_units"),
                activation='elu',
                input_shape=(self.num_train.shape[1],)
            )
        )
        model.add(Dropout(0.5))
        model.add(BatchNormalization())

        model.add(
            Dense(
                self.experiment.get_parameter("second_layer_units"),
                activation='elu'))

        model.add(Dropout(0.3))
        model.add(BatchNormalization())

        merged_out = keras.layers.concatenate([model.output, self.text_model.output])
        merged_out = keras.layers.Dense(
                        self.experiment.get_parameter("third_layer_units"), 
                        activation='elu',
                        kernel_initializer='HeNormal')(merged_out)

        merged_out = keras.layers.Dense(8, activation='softmax')(merged_out)
        
        combined_model = keras.Model([model.input] + [self.text_model.input], [merged_out])
        combined_model.compile(
            loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy']
        )
        return combined_model
    
    def train_model(self):
        # Train the model
        combined_model = self.build_model()
        model = ktrain.get_learner(
            combined_model,
            train_data=self.train_data,
            val_data=self.valid_data,
            batch_size=self.experiment.get_parameter("batch_size")
        )
        model.fit_onecycle(
            lr=0.001,
            epochs=self.experiment.get_parameter("epochs"),
            class_weight=self.class_weight,
        )

        return model
    
    def evaluate_model(self):
        # Evaluate the model
        model = self.train_model()
        score = model.evaluate(test_data=self.valid_data)
        # LOGGER.info(f"{ score }")
    
    def grid_search(self, config_dict):
        opt = comet_ml.Optimizer(config_dict)
        for self.experiment in opt.get_experiments(project_name="music-genre-multiclass-classification"):
            # self.experiment.log_parameters("epochs", 10)

            self.build_model()
            self.train_model()
            self.evaluate_model()
            self.experiment.end()

## Grid search
- using the comet_ml library, we can track the hyperparameters and metrics and all the metrics are on the comet_ml dashboard.

In [42]:
audio_model = AudioTextGenreClassifier(
    num_train,
    num_valid,
    train_data,
    num_train_y,
    valid_data,
    num_valid_y,
    class_weight,
    early_stop,
    experiment,
    model2,
)
# mu is the mean number of units, sigma is the standard deviation
audio_model.grid_search({
    "algorithm": "bayes",
    "name": "Optimize Music Classification Network",
    "spec": {"maxCombo": 10, "objective": "minimize", "metric": "loss"},
    "parameters": {
        "first_layer_units": {"type": "discrete", "values": [550,600,800,850,900,950,1000]},
        "second_layer_units": {"type": "discrete", "values": [100,150,200,225,250,275,300]},
        "third_layer_units": {"type": "discrete", "values": [32, 64]},
        "batch_size": {"type": "discrete", "values": [16,32]},
        "epochs": {"type": "discrete", "values": [9,10,11,12,13]},
    },
    "trials": 1,
})

COMET INFO: COMET_OPTIMIZER_ID=6821192df6104fef97f1a5063deaceda
COMET INFO: Using optimizer config: {'algorithm': 'bayes', 'configSpaceSize': 980, 'endTime': None, 'id': '6821192df6104fef97f1a5063deaceda', 'lastUpdateTime': None, 'maxCombo': 10, 'name': 'Optimize Music Classification Network', 'parameters': {'batch_size': {'type': 'discrete', 'values': [16, 32]}, 'epochs': {'type': 'discrete', 'values': [9, 10, 11, 12, 13]}, 'first_layer_units': {'type': 'discrete', 'values': [550, 600, 800, 850, 900, 950, 1000]}, 'second_layer_units': {'type': 'discrete', 'values': [100, 150, 200, 225, 250, 275, 300]}, 'third_layer_units': {'type': 'discrete', 'values': [32, 64]}}, 'predictor': None, 'spec': {'gridSize': 10, 'maxCombo': 10, 'metric': 'loss', 'minSampleSize': 100, 'objective': 'minimize', 'retryAssignLimit': 0, 'retryLimit': 1000}, 'startTime': 26853102834, 'state': {'mode': None, 'seed': None, 'sequence': [], 'sequence_i': 0, 'sequence_pid': None, 'sequence_retry': 0, 'sequence_retry_



begin training using onecycle policy with max lr of 0.001...
Epoch 1/9
240/240 [==============================] - 3s 8ms/step - loss: 1.9502 - accuracy: 0.3181 - val_loss: 1.4439 - val_accuracy: 0.4644
Epoch 2/9
240/240 [==============================] - 2s 6ms/step - loss: 1.5832 - accuracy: 0.4219 - val_loss: 1.3706 - val_accuracy: 0.5311
Epoch 3/9
240/240 [==============================] - 1s 6ms/step - loss: 1.4148 - accuracy: 0.4691 - val_loss: 1.3437 - val_accuracy: 0.5200
Epoch 4/9
240/240 [==============================] - 1s 6ms/step - loss: 1.3240 - accuracy: 0.5013 - val_loss: 1.1402 - val_accuracy: 0.6422
Epoch 5/9
240/240 [==============================] - 2s 6ms/step - loss: 1.2339 - accuracy: 0.5387 - val_loss: 1.1942 - val_accuracy: 0.5867
Epoch 6/9
240/240 [==============================] - 2s 6ms/step - loss: 1.1721 - accuracy: 0.5576 - val_loss: 1.1239 - val_accuracy: 0.5867
Epoch 7/9
240/240 [==============================] - 2s 6ms/step - loss: 1.1128 - accuracy:

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/28bf5de7e370492b9c0f5013d9dd0cbb
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [18]                : (0.3180515766143799, 0.6146131753921509)
COMET INFO:     batch_accuracy [432]         : (0.0625, 0.71875)
COMET INFO:     batch_loss [432]             : (0.7794969081878662, 3.2433366775512695)
COMET INFO:     epoch_duration [18]          : (1.469000000000051, 3.0780000000004293)
COMET INFO:     loss [18]                    : (1.0197501182556152, 1.9887447357177734)
COMET INFO:     val_accuracy [18]            : (0.46444445848464966, 0.644444465637207)
COMET INFO:     val_loss [18]                : (1.0267419815063477, 1.443869709968567)
COMET INFO:     validate_batch_accura



begin training using onecycle policy with max lr of 0.001...
Epoch 1/10
240/240 [==============================] - 3s 8ms/step - loss: 1.8696 - accuracy: 0.3603 - val_loss: 1.3881 - val_accuracy: 0.4778
Epoch 2/10
240/240 [==============================] - 2s 7ms/step - loss: 1.5017 - accuracy: 0.4513 - val_loss: 1.2807 - val_accuracy: 0.5378
Epoch 3/10
240/240 [==============================] - 2s 7ms/step - loss: 1.3665 - accuracy: 0.4988 - val_loss: 1.2808 - val_accuracy: 0.5444
Epoch 4/10
240/240 [==============================] - 2s 7ms/step - loss: 1.2742 - accuracy: 0.5257 - val_loss: 1.2171 - val_accuracy: 0.5889
Epoch 5/10
240/240 [==============================] - 2s 7ms/step - loss: 1.2112 - accuracy: 0.5527 - val_loss: 1.1930 - val_accuracy: 0.5667
Epoch 6/10
240/240 [==============================] - 2s 7ms/step - loss: 1.1262 - accuracy: 0.5854 - val_loss: 1.0451 - val_accuracy: 0.6267
Epoch 7/10
240/240 [==============================] - 2s 8ms/step - loss: 1.0710 - ac

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/d4727c3de0614b469b682e9c72f125ab
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [20]                : (0.3467048704624176, 0.6423547863960266)
COMET INFO:     batch_accuracy [480]         : (0.125, 0.75)
COMET INFO:     batch_loss [480]             : (0.6739035844802856, 2.7199299335479736)
COMET INFO:     epoch_duration [20]          : (1.6869999999998981, 2.717999999999847)
COMET INFO:     loss [20]                    : (0.9161579608917236, 1.8695846796035767)
COMET INFO:     val_accuracy [20]            : (0.4355555474758148, 0.6644444465637207)
COMET INFO:     val_loss [20]                : (0.9968975782394409, 1.5395829677581787)
COMET INFO:     validate_batch_accuracy 

...

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/7b44d79925f84b4e998372dc4c64354c
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [22]                : (0.34201616048812866, 0.7119041681289673)
COMET INFO:     batch_accuracy [528]         : (0.09375, 0.875)
COMET INFO:     batch_loss [528]             : (0.48772338032722473, 2.757594108581543)
COMET INFO:     epoch_duration [22]          : (1.6869999999998981, 2.82799999999952)
COMET INFO:     loss [22]                    : (0.716766357421875, 1.865480661392212)
COMET INFO:     val_accuracy [22]            : (0.47111111879348755, 0.6733333468437195)
COMET INFO:     val_loss [22]                : (1.0391194820404053, 1.4545435905456543)
COMET INFO:     validate_batch_accurac



begin training using onecycle policy with max lr of 0.001...
Epoch 1/13
480/480 [==============================] - 4s 8ms/step - loss: 1.9136 - accuracy: 0.3250 - val_loss: 1.4149 - val_accuracy: 0.5044
Epoch 2/13
480/480 [==============================] - 3s 7ms/step - loss: 1.5068 - accuracy: 0.4400 - val_loss: 1.2339 - val_accuracy: 0.5889
Epoch 3/13
480/480 [==============================] - 3s 7ms/step - loss: 1.2891 - accuracy: 0.5162 - val_loss: 1.1888 - val_accuracy: 0.5978
Epoch 4/13
480/480 [==============================] - 3s 7ms/step - loss: 1.1491 - accuracy: 0.5686 - val_loss: 1.0618 - val_accuracy: 0.6244
Epoch 5/13
480/480 [==============================] - 3s 7ms/step - loss: 1.0645 - accuracy: 0.5882 - val_loss: 1.0309 - val_accuracy: 0.6400
Epoch 6/13
480/480 [==============================] - 3s 7ms/step - loss: 1.0081 - accuracy: 0.6002 - val_loss: 1.0887 - val_accuracy: 0.6111
Epoch 7/13
480/480 [==============================] - 3s 7ms/step - loss: 1.0065 - ac

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/383fae3dffcb49f08c31515d5a43bc43
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [26]                : (0.32495442032814026, 0.68442302942276)
COMET INFO:     batch_accuracy [1248]        : (0.0, 0.75)
COMET INFO:     batch_loss [1248]            : (0.5381028056144714, 3.60591983795166)
COMET INFO:     epoch_duration [26]          : (3.219000000000051, 4.344000000000051)
COMET INFO:     loss [26]                    : (0.7689998149871826, 1.913564920425415)
COMET INFO:     val_accuracy [26]            : (0.4933333396911621, 0.6555555462837219)
COMET INFO:     val_loss [26]                : (1.0308988094329834, 1.441412329673767)
COMET INFO:     validate_batch_accuracy [52] : (



begin training using onecycle policy with max lr of 0.001...
Epoch 1/11
240/240 [==============================] - 3s 8ms/step - loss: 1.9547 - accuracy: 0.3204 - val_loss: 1.4694 - val_accuracy: 0.4867
Epoch 2/11
240/240 [==============================] - 2s 7ms/step - loss: 1.5137 - accuracy: 0.4396 - val_loss: 1.3196 - val_accuracy: 0.5533
Epoch 3/11
240/240 [==============================] - 2s 7ms/step - loss: 1.3115 - accuracy: 0.5186 - val_loss: 1.2579 - val_accuracy: 0.5622
Epoch 4/11
240/240 [==============================] - 2s 7ms/step - loss: 1.1477 - accuracy: 0.5745 - val_loss: 1.0799 - val_accuracy: 0.6467
Epoch 5/11
240/240 [==============================] - 2s 7ms/step - loss: 1.0410 - accuracy: 0.6029 - val_loss: 1.1244 - val_accuracy: 0.6356
Epoch 6/11
240/240 [==============================] - 2s 7ms/step - loss: 0.9573 - accuracy: 0.6319 - val_loss: 1.0837 - val_accuracy: 0.6133
Epoch 7/11
240/240 [==============================] - 2s 7ms/step - loss: 0.8791 - ac

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/35d3346e0f0a482591139103b91c36f3
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [22]                : (0.32039594650268555, 0.7061734795570374)
COMET INFO:     batch_accuracy [528]         : (0.0, 0.9375)
COMET INFO:     batch_loss [528]             : (0.5200537443161011, 2.927767753601074)
COMET INFO:     epoch_duration [22]          : (1.7030000000004293, 2.969000000000051)
COMET INFO:     loss [22]                    : (0.698472797870636, 1.9547213315963745)
COMET INFO:     val_accuracy [22]            : (0.4866666793823242, 0.6466666460037231)
COMET INFO:     val_loss [22]                : (1.0799001455307007, 1.4693771600723267)
COMET INFO:     validate_batch_accuracy [

# Build and save the best model for prediction

In [51]:
model = Sequential()
model.add(
    Dense(
        850,
        activation='elu',
        input_shape=(num_train.shape[1],)
    )
)
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(
    Dense(
        275,
        activation='elu'))

model.add(Dropout(0.3))
model.add(BatchNormalization())

merged_out = keras.layers.concatenate([model.output, model2.output])
merged_out = keras.layers.Dense(
                64, 
                activation='elu',
                kernel_initializer='HeNormal')(merged_out)

merged_out = keras.layers.Dense(8, activation='softmax')(merged_out)

combined_model = keras.Model([model.input] + [model2.input], [merged_out])
combined_model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [52]:
learner = ktrain.get_learner(
            combined_model,
            train_data=train_data,
            val_data=valid_data,
            batch_size=32
        )
learner.fit_onecycle(
    lr=0.001,
    epochs=11,
    class_weight=class_weight,
)



begin training using onecycle policy with max lr of 0.001...
Epoch 1/11
240/240 [==============================] - 2s 7ms/step - loss: 1.8960 - accuracy: 0.3463 - val_loss: 1.5080 - val_accuracy: 0.4133
Epoch 2/11
240/240 [==============================] - 2s 6ms/step - loss: 1.4687 - accuracy: 0.4604 - val_loss: 1.3603 - val_accuracy: 0.4867
Epoch 3/11
240/240 [==============================] - 1s 6ms/step - loss: 1.2600 - accuracy: 0.5283 - val_loss: 1.2178 - val_accuracy: 0.5822
Epoch 4/11
240/240 [==============================] - 2s 7ms/step - loss: 1.1250 - accuracy: 0.5834 - val_loss: 1.2358 - val_accuracy: 0.5822
Epoch 5/11
240/240 [==============================] - 1s 6ms/step - loss: 0.9789 - accuracy: 0.6285 - val_loss: 1.1996 - val_accuracy: 0.5822
Epoch 6/11
240/240 [==============================] - 1s 6ms/step - loss: 0.9285 - accuracy: 0.6425 - val_loss: 1.2572 - val_accuracy: 0.5689
Epoch 7/11
240/240 [==============================] - 2s 7ms/step - loss: 0.8549 - ac

In [53]:
predictor = ktrain.get_predictor(model=learner.model, preproc=preproc2)

In [54]:
predictor.save('audio_text_model')